In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

import os
import csv
import numpy as np

from PIL import Image

## Discover dataset structure

Create image and annotation lists:

In [ ]:
root = '/data1/joan/eurus/data/alov300/'

img_path = os.path.join(root, 'images')
ann_path = os.path.join(root, 'annotations')

print('Image path: ', img_path)
print('Annotations path: ', ann_path)

In [ ]:
sequences = next(os.walk(img_path))[1]

print('Sequences: ')
print(sequences)

In [ ]:
ann_list = []
img_list = []

for seq in sequences:
    ann_path2, _, files = next(os.walk(os.path.join(ann_path, seq)))
    files = sorted(files)
    ann_list2 = []
    ann_indices = []
    for file in files:
        with open(os.path.join(ann_path2, file), "r") as f:
            truths = list(csv.reader(f, delimiter=' '))
        ann_list3 = []
        indices = []
        for t in truths:
            xs = np.array(t[1::2]).astype(np.float32)
            ys = np.array(t[2::2]).astype(np.float32)
            tl = np.array([np.min(xs), np.min(ys)])
            br = np.array([np.max(xs), np.max(ys)])
            sz = br - tl
            ann_list3.append(np.concatenate([tl, sz]))
            indices.append(int(t[0]) - 1)
        ann_list2.append(ann_list3)
        ann_indices.append(indices)
    ann_list.append(ann_list2)
    
    
    img_path2, dirs, _ = next(os.walk(os.path.join(img_path, seq)))
    dirs = sorted(dirs)
    
    img_list2 = []
    for d, indices in zip(dirs, ann_indices):
        img_path3, _, files = next(os.walk(os.path.join(img_path2, d)))
        files = sorted(files)
        files = [files[i] for i in indices]
        img_list3 = []
        for file in files:
            img_list3.append(os.path.join(img_path3, file))    
        img_list2.append(img_list3)
    img_list.append(img_list2) 

Make sure every image is associated to its corresponding annotations:

In [ ]:
n_ann = 0

print('Annotations tree: ')
print('  - ' + str(len(ann_list)))
for a in ann_list:
    print('    - '+ str(len(a)))
    for b in a:
        print('      - '+ str(len(b)))
        n_ann += len(b)

In [ ]:
print('Number of annotations: ', n_ann)

In [ ]:
n_img = 0

print('Images tree: ')
print('  - ' + str(len(img_list)))
for a in img_list:
    print('    - '+ str(len(a)))
    for b in a:
        print('      - '+ str(len(b)))
        n_img += len(b)

In [ ]:
print('Number of images: ', n_img)

## Visualize

Visualize a random image with its annotations:

In [ ]:
ind = np.random.randint(len(img_list))
ind2 = np.random.randint(len(img_list[ind]))
ind3 = np.random.randint(len(img_list[ind][ind2]))

img_path = img_list[ind][ind2][ind3]
img = Image.open(img_path)

ann = ann_list[ind][ind2][ind3]


fig = plt.figure(figsize=(32, 24))
ax = fig.add_subplot(111, aspect='equal')
ax.set_axis_off()
ax.imshow(img)

ax.add_patch(Rectangle((ann[0], ann[1]), ann[2], ann[3], alpha=.5))

## Dataset class

Prototype eurus' `Alov300` dataset class:

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

import os
import sys
import csv
import numbers

import numpy as np

from PIL import Image
from torch.utils.data import Dataset

from eurus.track.pytorch.model.dataset.utils import display_image

from ipywidgets import interact
import ipywidgets as widgets

In [ ]:
class Alov300(Dataset):
    r"""
    Class for the Amsterdam Library of Ordinary Videos (ALOV) 300 dataset.

    Parameters
    ----------
    root : str
        The root path to the dataset.
    transform :

    target_transform :

    context_factor : float, optional

    search_factor : float, optional

    context_size : int, optional

    search_size : int, optional


    References
    ----------
    A. W. Smeulders, et al. Visual tracking: An experimental survey.
    TPAMI 2013.
    """
    def __init__(self, root, transform=None, target_transform=None,
                 context_factor=3.0, search_factor=2.0,
                 context_size=128, search_size=256):

        self.root = root
        self.transform = transform
        self.target_transform = target_transform
        self.context_factor = context_factor
        self.search_factor = search_factor

        if isinstance(context_size, numbers.Number):
            self.context_size = np.array([context_size, context_size]).astype(
                np.int32)
        elif isinstance(context_size, tuple):
            self.context_size = np.array(context_size)
        else:
            raise ValueError('`context_size` must be `int` or `(int, int)`.')

        if isinstance(search_size, numbers.Number):
            self.search_size = np.array([search_size, search_size]).astype(
                np.int32)
        elif isinstance(search_size, tuple):
            self.search_size = np.array(search_size).astype(np.int32)
        else:
            raise ValueError('`context_size` must be `int` or `(int, int)`.')

        img_path = os.path.join(root, 'images')
        ann_path = os.path.join(root, 'annotations')

        sequences = next(os.walk(img_path))[1]

        self.ann_list = []
        self.img_list = []

        for seq in sequences:
            ann_path2, _, files = next(os.walk(os.path.join(ann_path, seq)))
            files = sorted(files)
            ann_list2 = []
            ann_indices = []
            for file in files:
                with open(os.path.join(ann_path2, file), "r") as f:
                    truths = list(csv.reader(f, delimiter=' '))
                ann_list3 = []
                indices = []
                for t in truths:
                    xs = np.array(t[1::2]).astype(np.float32)
                    ys = np.array(t[2::2]).astype(np.float32)
                    tl = np.array([np.min(xs), np.min(ys)])
                    br = np.array([np.max(xs), np.max(ys)])
                    sz = br - tl
                    ann_list3.append(np.concatenate([tl, sz]))
                    indices.append(int(t[0]) - 1)
                ann_list2.append(ann_list3)
                ann_indices.append(indices)
            self.ann_list.append(ann_list2)

            img_path2, dirs, _ = next(os.walk(os.path.join(img_path, seq)))
            dirs = sorted(dirs)
            img_list2 = []
            for d, indices in zip(dirs, ann_indices):
                img_path3, _, files = next(os.walk(os.path.join(img_path2, d)))
                files = sorted(files)
                files = [files[i] for i in indices]
                img_list3 = []
                for file in files:
                    img_list3.append(os.path.join(img_path3, file))
                img_list2.append(img_list3)
            self.img_list.append(img_list2)

        assert len(self.img_list) == len(self.ann_list), \
            'The number of image ({}) and ann ({}) groups should be ' \
            'the same.'.format(len(self.img_list), len(self.ann_list))
        for i, (img_list2, ann_list2) in enumerate(zip(self.img_list,
                                                       self.ann_list)):
            assert len(img_list2) == len(ann_list2), \
                'The number of image ({}) and annotations ({}) sequences ' \
                'in group {} should be the same.'.format(
                    len(img_list2), len(ann_list2), i)
            for j, (img_list3, ann_list3) in enumerate(zip(img_list2,
                                                           ann_list2)):
                assert len(img_list3) == len(ann_list3), \
                    'The number of image ({}) and annotations ({}) in ' \
                    'sequence {} of group {} should be the same.'.format(
                        len(img_list3), len(ann_list3), j, i)
                    
    def view(self):
        r"""
        Visualize dataset.
        """
        if 'ipykernel' in sys.modules:
            self._notebook_view()
        else:
            print("hello")

    def _notebook_view(self):
        def _view_image(group_index, sequence_index, index):
            img = Image.open(self.img_list[group_index][sequence_index][index])
            ann = self.ann_list[group_index][sequence_index][index]
            display_image(img, ann)

        group_slider = widgets.IntSlider(
            value=0,
            min=0,
            max=len(self.img_list) - 1,
            step=1,
            description='group:\t',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='i',
            slider_color='white'
        )

        sequence_slider = widgets.IntSlider(
            value=0,
            min=0,
            max=len(self.img_list[0]) - 1,
            step=1,
            description='sequence: \t',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='i',
            slider_color='white'
        )

        slider = widgets.IntSlider(
            value=0,
            min=0,
            max=len(self.img_list[0][0]) - 1,
            step=1,
            description='image: \t',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='i',
            slider_color='white'
        )

        def update_sequence_range(*args):
            i = group_slider.value
            j = sequence_slider.value
            slider.max = len(self.img_list[i][j]) - 1
            slider.value = 0
            
        sequence_slider.observe(update_sequence_range)
            
        def update_group_range(*args):
            i = group_slider.value
            j = sequence_slider.value
            sequence_slider.max = len(self.img_list[i]) - 1
            sequence_slider.value = 0
            slider.max = len(self.img_list[i][j]) - 1
            slider.value = 0
                
        group_slider.observe(update_group_range)

        interact(_view_image,
                 group_index=group_slider,
                 sequence_index=sequence_slider,
                 index=slider)

In [ ]:
import matplotlib.patches as patches

def display_image(img, ann, figure_size=(32, 24), color='red', fill=False,
                  alpha=1.0):
    r"""
    Display image with overlaid bounding box.

    Parameters
    ----------
    img : np.ndarray
        ``(height, width, 3)`` array containing the image to be displayed.
    ann : np.ndarray
        ``(4, )`` array containing the top left vertex and the size of the
        bounding box to be displayed.
    figure_size : tuple(int), optional
        ``(2,)`` tuple defining the figure size.
    color : 'str'

    fill : bool

    alpha: float

    """
    fig, ax = plt.subplots(1, figsize=(16, 9))
    ax.set_axis_off()

    ax.imshow(img)

    box = patches.Rectangle((ann[0], ann[1]), ann[2], ann[3],
                            color=color, fill=fill, alpha=alpha)
    ax.add_patch(box)
    plt.show()

Use dataset class:

In [ ]:
dataset = Alov300('/data1/joan/eurus/data/alov300/')

Use widget:

In [ ]:
dataset.view()

## Alov300 Eurus Class

In [ ]:
from eurus.track.pytorch.model.dataset import Alov300
from eurus.track.pytorch.model.dataset.utils import display_image, display_search_image

Create dataset instance:

In [ ]:
dataset = Alov300('/data1/joan/eurus/data/alov300/')

In [ ]:
print('The alov300 dataset contains:', len(dataset), 'sequences')

In [ ]:
print(dataset)

In [ ]:
dataset.view_sequence_length_histogram()

View original unprocessed dataset:

In [ ]:
dataset.view_original()

Make sure index work as expected:

In [ ]:
gen = dataset[304]

In [ ]:
context, search, context_box, search_box, prior_search_box = next(gen)

In [ ]:
display_image(context, context_box)

In [ ]:
display_search_image(search, search_box, prior_search_box)